### Copy PyTorch cifar10 model to MINIO
Make sure MINIO Setup is done, Refer [minio_setup.ipynb](./minio_setup.ipynb)

In [ ]:
%%bash
kubectl cluster-info --context kind-seldon
mc config host add gcs https://storage.googleapis.com "" "" 

mc mb minio-seldon/pytorchcifar10 -p
mc cp gcs/seldon-models/triton/pytorch_cifar10/cifar10 minio-seldon/pytorchcifar10/ --recursive

#### Create namespace for experimentation

In [ ]:
!kubectl create namespace seldon
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

#### TorchScript CIFAR10 Model


In [1]:
%%writefile secret.yaml
apiVersion: v1
kind: Secret
metadata:
  name: seldon-init-pt-secret
type: Opaque
stringData:
  RCLONE_CONFIG_S3_TYPE: s3
  RCLONE_CONFIG_S3_PROVIDER: minio
  RCLONE_CONFIG_S3_ACCESS_KEY_ID: minioadmin
  RCLONE_CONFIG_S3_SECRET_ACCESS_KEY: minioadmin
  RCLONE_CONFIG_S3_ENDPOINT: http://minio.minio-system.svc.cluster.local:9000
  RCLONE_CONFIG_S3_ENV_AUTH: "false"

Overwriting secret.yaml


In [2]:
!kubectl delete -f secret.yaml
!kubectl create -f secret.yaml

Error from server (NotFound): error when deleting "secret.yaml": secrets "seldon-init-pt-secret" not found
secret/seldon-init-pt-secret created


In [3]:
%%writefile resources/triton_pt_cifar10.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: cifar10
  namespace: seldon
spec:
  annotations:
    seldon.io/engine-seldon-log-messages-externally: "true"
  name: resnet32
  predictors:
  - graph:
      implementation: TRITON_SERVER
      logger:
        mode: all
      modelUri: s3://pytorchcifar10
      envSecretRefName: seldon-init-pt-secret      
      name: cifar10
      type: MODEL
    name: default
    replicas: 1
  protocol: v2

Overwriting resources/triton_pt_cifar10.yaml


In [4]:
!kubectl delete -f resources/triton_pt_cifar10.yaml
!kubectl create -f resources/triton_pt_cifar10.yaml

error: error parsing resources/triton_pt_cifar10.yaml: error converting YAML to JSON: yaml: line 13: mapping values are not allowed in this context
error: error parsing resources/triton_pt_cifar10.yaml: error converting YAML to JSON: yaml: line 13: mapping values are not allowed in this context
